# LL for an input Image

In [1]:
# imports
from importlib import reload
import os
#
import torch

from ulmo import ood
from ulmo.models import autoencoders, flows, ConditionalFlow

# Init

## Device

In [2]:
device = torch.device('cpu')

In [3]:
model_path = '/home/xavier/Projects/Oceanography/AI/ODD/2010_128x128/'
d_path = '/home/xavier/Projects/Oceanography/AI/ODD/'

# Load up the model

## DCAE

In [4]:
reload(autoencoders)
dcae = autoencoders.DCAE.from_file(os.path.join(model_path, 'autoencoder.pt'), image_shape=(1, 64, 64), latent_dim=512)

In [5]:
dcae

DCAE(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
  )
  (to_z): Lin

## Flow

### Instantiate

In [6]:
flow = ConditionalFlow(
    dim=512, 
    context_dim=None, 
    transform_type='autoregressive', 
    n_layers=10, 
    hidden_units=256,
    n_blocks=2, 
    dropout=0.2, 
    use_batch_norm=False, 
    tails='linear', 
    tail_bound=10, 
    n_bins=5,
    min_bin_height=1e-3, 
    min_bin_width=1e-3, 
    min_derivative=1e-3, 
    unconditional_transform=False, 
    encoder=None)

### Load

In [7]:
flow.load_state_dict(torch.load(os.path.join(model_path, 'flow.pt'), map_location=device))

<All keys matched successfully>

# Build the OOD

In [8]:
pae = ood.ProbabilisticAutoencoder(dcae, flow, 'tmp/', device=device)

In [9]:
pae.device

device(type='cpu')

# Give it a whirl!

In [10]:
# Compute log probs of fields in new HDF5 file
pae.compute_log_probs(
    input_file=os.path.join(d_path, 'MODIS_2010_95clear_128x128_small_train_iterative.h5'),
    dataset='train',
    output_file='./sample_output.h5',
    scaler=None)

RuntimeError: No scaler found or provided.